In [1]:
from dotenv import load_dotenv
import pandas as pd
import os
import base64
import requests

IMAGE_INPUT_FOLDER = "dataset/"

In [2]:
PROMPT = """
Q: 
Given a screenshot of a mobile page. Provide a short description of the pages intent. your description has to striclty follow this format:
"A page from {{What type of App the page is from}} presenting {{What is the main intent of the app?}}"
A:
A page from a language Learning application, presenting a sentence translation task for the user.

Q: 
Given a screenshot of a mobile page. Provide a short description of the pages intent. your description has to striclty follow this format:
"A page from a {{What type of App the page is from}} presenting {{What is the main intent of the app?}}"
A:
A page from an e-commerce app, presenting a List of armchairs to buy.

Q: 
Given a screenshot of a mobile page. Provide a short description of the pages intent. your description has to striclty follow this format:
"A page from a {{What type of App the page is from}} presenting {{What is the main intent of the app?}}"

A:
"""

In [3]:
def encode_image(id):
    image_path = IMAGE_INPUT_FOLDER + f"{id}.jpg"
    with open(image_path,"rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')    
   

In [4]:
def describe_image(id):
    api_key = os.getenv("OPENAI_API_KEY")
    base64_image = encode_image(id)

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }

    payload = {
        "model": "gpt-4-vision-preview",
        "messages": [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": 
                    f"{PROMPT}"
                }
            ]
        },
        {
            "role": "user",
            "content": [
            {
                "type": "image_url",
                "image_url": {
                "url": f"data:image/jpeg;base64,{base64_image}"
                }
            }
            ]
        }
        ],
        "max_tokens": 300
    }
    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
    return response